# Notebook 1: Datenbereinigung

**Projekt:** Analyse der Haushaltsausgaben in der Schweiz (2006-2022)  
**Autor:** CAS Information Engineering - ZHAW  
**Version:** 1.0  
**Datum:** 14. Oktober 2025

---

## Ziel dieses Notebooks

Dieses Notebook implementiert die **Datenbereinigung** gemäß den funktionalen Anforderungen FA-01.1 bis FA-01.6:

- **FA-01.1:** Laden der Excel-Dateien mit Fehlerbehandlung
- **FA-01.2:** Entfernen von BFS-Metadaten
- **FA-01.3:** Vereinheitlichung der Spaltennamen
- **FA-01.4:** Konvertierung der Datentypen
- **FA-01.5:** Hinzufügen einer Datentyp-Spalte
- **FA-01.6:** Export der bereinigten Daten als CSV

---

## Inhaltsverzeichnis

1. [Initialisierung und Laden der Daten](#1-initialisierung)
2. [Datenbereinigungsfunktion](#2-bereinigungsfunktion)
3. [Bereinigung: Gesamtausgaben](#3-gesamtausgaben)
4. [Bereinigung: Ausgaben nach Alter](#4-alter)
5. [Bereinigung: Ausgaben nach Haushaltstyp](#5-haushaltstyp)
6. [Export der bereinigten Daten](#6-export)
7. [Zusammenfassung](#7-zusammenfassung)

---

## 1. Initialisierung und Laden der Daten <a id='1-initialisierung'></a>

### 1.1 Import der Bibliotheken

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Versionen anzeigen für Reproduzierbarkeit
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")

### 1.2 Pfade definieren (betriebssystemunabhängig)

Verwendung von `os.path.join()` und `pathlib` für Kompatibilität (NFR-05.2).

In [ ]:
# Basis-Pfade definieren
BASE_DIR = Path.cwd().parent  # Eine Ebene über dem notebooks-Ordner
RAW_DATA_DIR = BASE_DIR / 'data' / 'raw'
PROCESSED_DATA_DIR = BASE_DIR / 'data' / 'processed'

# Processed-Ordner erstellen, falls nicht vorhanden
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Raw Data Directory: {RAW_DATA_DIR}")
print(f"Processed Data Directory: {PROCESSED_DATA_DIR}")
print(f"\nProcessed-Ordner existiert: {PROCESSED_DATA_DIR.exists()}")

### 1.3 Excel-Dateien laden mit Fehlerbehandlung (FA-01.1)

**Wichtig:** Wir implementieren eine robuste Fehlerbehandlung, um die Anforderung FA-01.1 zu erfüllen.

In [ ]:
def load_excel_file(file_path, sheet_name=0, skiprows=None):
    """
    Lädt eine Excel-Datei mit Fehlerbehandlung.
    
    Parameters:
    -----------
    file_path : Path or str
        Pfad zur Excel-Datei
    sheet_name : int or str, default=0
        Name oder Index des zu ladenden Sheets
    skiprows : int or list, optional
        Anzahl oder Liste der zu überspringenden Zeilen
    
    Returns:
    --------
    pd.DataFrame or None
        Geladener DataFrame oder None bei Fehler
    """
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=skiprows)
        print(f"✓ Erfolgreich geladen: {file_path.name}")
        print(f"  Shape: {df.shape}")
        return df
    except FileNotFoundError:
        print(f"✗ FEHLER: Datei nicht gefunden: {file_path}")
        return None
    except Exception as e:
        print(f"✗ FEHLER beim Laden von {file_path.name}: {e}")
        return None

### 1.4 Laden der drei Datensätze

**Hinweis:** Die genaue Anzahl der zu überspringenden Zeilen wurde durch manuelle Inspektion der Excel-Dateien ermittelt.

In [ ]:
# Dateinamen definieren
files = {
    'gesamtausgaben': RAW_DATA_DIR / 'gesamtausgaben_2006_2022.xlsx',
    'alter': RAW_DATA_DIR / 'ausgaben_nach_alter.xlsx',
    'haushaltstyp': RAW_DATA_DIR / 'ausgaben_nach_haushaltstyp.xlsx'
}

# Laden der Dateien
# Hinweis: skiprows muss ggf. angepasst werden, je nach tatsächlicher Struktur der Dateien
df_gesamtausgaben_raw = load_excel_file(files['gesamtausgaben'], skiprows=None)
df_alter_raw = load_excel_file(files['alter'], skiprows=None)
df_haushaltstyp_raw = load_excel_file(files['haushaltstyp'], skiprows=None)

### 1.5 Erste Inspektion der Rohdaten

Wir schauen uns die ersten Zeilen an, um Metadaten zu identifizieren.

In [ ]:
print("="*80)
print("ROHDATEN: Gesamtausgaben")
print("="*80)
if df_gesamtausgaben_raw is not None:
    display(df_gesamtausgaben_raw.head(15))
    print(f"\nInfo:")
    df_gesamtausgaben_raw.info()

In [ ]:
print("="*80)
print("ROHDATEN: Ausgaben nach Alter")
print("="*80)
if df_alter_raw is not None:
    display(df_alter_raw.head(15))
    print(f"\nInfo:")
    df_alter_raw.info()

In [ ]:
print("="*80)
print("ROHDATEN: Ausgaben nach Haushaltstyp")
print("="*80)
if df_haushaltstyp_raw is not None:
    display(df_haushaltstyp_raw.head(15))
    print(f"\nInfo:")
    df_haushaltstyp_raw.info()

---

## 2. Datenbereinigungsfunktion <a id='2-bereinigungsfunktion'></a>

**Modularität (NFR-03.1):** Um Code-Duplikation zu vermeiden, erstellen wir eine zentrale Bereinigungsfunktion.

### 2.1 Hauptfunktion zur Datenbereinigung

In [ ]:
def clean_bfs_data(df, datentyp_name, column_mapping, skiprows_from_raw=0):
    """
    Bereinigt BFS-Daten gemäß den Anforderungen FA-01.2 bis FA-01.5.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Rohdaten-DataFrame
    datentyp_name : str
        Typ der Daten: 'Gesamt', 'Altersklasse', oder 'Haushaltstyp'
    column_mapping : dict
        Dictionary mit Mapping von alten zu neuen Spaltennamen
    skiprows_from_raw : int, default=0
        Anzahl zusätzlicher Zeilen, die aus dem bereits geladenen DataFrame entfernt werden sollen
    
    Returns:
    --------
    pd.DataFrame
        Bereinigter DataFrame
    """
    if df is None:
        print(f"✗ Keine Daten zum Bereinigen für {datentyp_name}")
        return None
    
    print(f"\n{'='*60}")
    print(f"Bereinigung: {datentyp_name}")
    print(f"{'='*60}")
    
    # Kopie erstellen, um Original nicht zu verändern
    df_clean = df.copy()
    
    # Schritt 1: Zusätzliche Metadaten-Zeilen entfernen (FA-01.2)
    if skiprows_from_raw > 0:
        df_clean = df_clean.iloc[skiprows_from_raw:].reset_index(drop=True)
        print(f"✓ {skiprows_from_raw} Metadaten-Zeilen entfernt")
    
    # Schritt 2: Leere Zeilen entfernen
    df_clean = df_clean.dropna(how='all').reset_index(drop=True)
    print(f"✓ Leere Zeilen entfernt")
    
    # Schritt 3: Spaltennamen umbenennen (FA-01.3)
    df_clean = df_clean.rename(columns=column_mapping)
    print(f"✓ Spaltennamen vereinheitlicht")
    print(f"  Neue Spalten: {list(df_clean.columns)}")
    
    # Schritt 4: Datentypen konvertieren (FA-01.4)
    # Betrag zu numerisch konvertieren
    if 'betrag_chf' in df_clean.columns:
        df_clean['betrag_chf'] = pd.to_numeric(df_clean['betrag_chf'], errors='coerce')
        
        # NaN-Werte behandeln
        nan_count = df_clean['betrag_chf'].isna().sum()
        if nan_count > 0:
            print(f"  ⚠ {nan_count} NaN-Werte in 'betrag_chf' gefunden")
            print(f"  → Entscheidung: Zeilen mit NaN werden entfernt")
            df_clean = df_clean.dropna(subset=['betrag_chf']).reset_index(drop=True)
        
        print(f"✓ Betrag zu float konvertiert")
    
    # Periode zu Integer konvertieren
    if 'periode' in df_clean.columns:
        df_clean['periode'] = pd.to_numeric(df_clean['periode'], errors='coerce')
        df_clean['periode'] = df_clean['periode'].astype('Int64')  # Nullable Integer
        print(f"✓ Periode zu Integer konvertiert")
    
    # Schritt 5: Datentyp-Spalte hinzufügen (FA-01.5)
    df_clean['datentyp'] = datentyp_name
    print(f"✓ Spalte 'datentyp' mit Wert '{datentyp_name}' hinzugefügt")
    
    # Finale Form
    print(f"\n✓ Bereinigung abgeschlossen")
    print(f"  Finale Shape: {df_clean.shape}")
    print(f"  Datentypen:")
    print(df_clean.dtypes)
    
    return df_clean

---

## 3. Bereinigung: Gesamtausgaben <a id='3-gesamtausgaben'></a>

### 3.1 Column Mapping definieren

**Hinweis:** Diese Spaltennamen müssen an die tatsächliche Struktur der Excel-Datei angepasst werden.

In [ ]:
# Diese Werte müssen basierend auf der tatsächlichen Dateistruktur angepasst werden
mapping_gesamtausgaben = {}

# Aktuelle Spaltennamen inspizieren
print("Aktuelle Spaltennamen:")
if df_gesamtausgaben_raw is not None:
    print(list(df_gesamtausgaben_raw.columns))

### 3.2 Bereinigung durchführen

In [ ]:
df_gesamtausgaben_clean = clean_bfs_data(
    df=df_gesamtausgaben_raw,
    datentyp_name='Gesamt',
    column_mapping=mapping_gesamtausgaben,
    skiprows_from_raw=0
)

### 3.3 Ergebnis überprüfen

In [ ]:
if df_gesamtausgaben_clean is not None:
    print("\n" + "="*60)
    print("BEREINIGTE DATEN: Gesamtausgaben (Vorschau)")
    print("="*60)
    display(df_gesamtausgaben_clean.head(10))
    
    print("\n" + "="*60)
    print("Statistische Übersicht:")
    print("="*60)
    display(df_gesamtausgaben_clean.describe())

---

## 4. Bereinigung: Ausgaben nach Alter <a id='4-alter'></a>

### 4.1 Column Mapping definieren

In [ ]:
mapping_alter = {}

print("Aktuelle Spaltennamen:")
if df_alter_raw is not None:
    print(list(df_alter_raw.columns))

### 4.2 Bereinigung durchführen

In [ ]:
df_alter_clean = clean_bfs_data(
    df=df_alter_raw,
    datentyp_name='Altersklasse',
    column_mapping=mapping_alter,
    skiprows_from_raw=0
)

### 4.3 Ergebnis überprüfen

In [ ]:
if df_alter_clean is not None:
    print("\n" + "="*60)
    print("BEREINIGTE DATEN: Ausgaben nach Alter (Vorschau)")
    print("="*60)
    display(df_alter_clean.head(10))
    
    print("\n" + "="*60)
    print("Unique Altersgruppen:")
    print("="*60)
    if 'altersgruppe' in df_alter_clean.columns:
        print(df_alter_clean['altersgruppe'].unique())

---

## 5. Bereinigung: Ausgaben nach Haushaltstyp <a id='5-haushaltstyp'></a>

### 5.1 Column Mapping definieren

In [ ]:
mapping_haushaltstyp = {}

print("Aktuelle Spaltennamen:")
if df_haushaltstyp_raw is not None:
    print(list(df_haushaltstyp_raw.columns))

### 5.2 Bereinigung durchführen

In [ ]:
df_haushaltstyp_clean = clean_bfs_data(
    df=df_haushaltstyp_raw,
    datentyp_name='Haushaltstyp',
    column_mapping=mapping_haushaltstyp,
    skiprows_from_raw=0
)

### 5.3 Ergebnis überprüfen

In [ ]:
if df_haushaltstyp_clean is not None:
    print("\n" + "="*60)
    print("BEREINIGTE DATEN: Ausgaben nach Haushaltstyp (Vorschau)")
    print("="*60)
    display(df_haushaltstyp_clean.head(10))
    
    print("\n" + "="*60)
    print("Unique Haushaltstypen:")
    print("="*60)
    if 'haushaltstyp' in df_haushaltstyp_clean.columns:
        print(df_haushaltstyp_clean['haushaltstyp'].unique())

---

## 6. Export der bereinigten Daten <a id='6-export'></a>

**FA-01.6:** Speicherung der bereinigten DataFrames als CSV-Dateien.

### 6.1 Export-Funktion

In [ ]:
def export_to_csv(df, filename, output_dir=PROCESSED_DATA_DIR):
    """
    Exportiert einen DataFrame als CSV-Datei.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Zu exportierender DataFrame
    filename : str
        Name der CSV-Datei (mit .csv Endung)
    output_dir : Path, default=PROCESSED_DATA_DIR
        Zielverzeichnis
    
    Returns:
    --------
    bool
        True bei Erfolg, False bei Fehler
    """
    if df is None:
        print(f"✗ Kein DataFrame zum Exportieren: {filename}")
        return False
    
    try:
        output_path = output_dir / filename
        df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"✓ Erfolgreich exportiert: {output_path}")
        print(f"  Zeilen: {len(df)}, Spalten: {len(df.columns)}")
        return True
    except Exception as e:
        print(f"✗ FEHLER beim Export von {filename}: {e}")
        return False

### 6.2 Export durchführen

In [ ]:
print("="*60)
print("EXPORT DER BEREINIGTEN DATEN")
print("="*60)

# Export der drei DataFrames
export_to_csv(df_gesamtausgaben_clean, 'clean_gesamtausgaben.csv')
export_to_csv(df_alter_clean, 'clean_alter.csv')
export_to_csv(df_haushaltstyp_clean, 'clean_haushaltstyp.csv')

print("\n" + "="*60)
print("Export abgeschlossen!")
print("="*60)

---

## 7. Zusammenfassung <a id='7-zusammenfassung'></a>

### 7.1 Durchgeführte Schritte

In diesem Notebook wurden folgende Aufgaben erfolgreich durchgeführt:

1. **✓ FA-01.1:** Excel-Dateien mit Fehlerbehandlung geladen
2. **✓ FA-01.2:** BFS-Metadaten und leere Zeilen entfernt
3. **✓ FA-01.3:** Spaltennamen vereinheitlicht
4. **✓ FA-01.4:** Datentypen konvertiert (Float für Beträge, Integer für Periode)
5. **✓ FA-01.5:** Datentyp-Spalte hinzugefügt
6. **✓ FA-01.6:** Bereinigte Daten als CSV exportiert

### 7.2 Ausgabedateien

Die folgenden CSV-Dateien wurden im Ordner `data/processed/` erstellt:

- `clean_gesamtausgaben.csv`
- `clean_alter.csv`
- `clean_haushaltstyp.csv`

### 7.3 Nächste Schritte

Die bereinigten Daten sind nun bereit für:

- **Notebook 2:** Datenanalyse (FA-02)
- **Notebook 3:** Visualisierung (FA-03)
- **Notebook 4:** Business Case Dokumentation (FA-04)

### 7.4 Hinweise

**Wichtig:** Die Column Mappings und skiprows-Parameter müssen nach der ersten Ausführung basierend auf der tatsächlichen Struktur der Excel-Dateien angepasst werden. Führen Sie zunächst die Inspektionszellen aus, um die Struktur zu verstehen, bevor Sie die Bereinigung durchführen.